In [1]:
import pandas as pd
import sqlite3

from datetime import datetime

In [2]:
con = sqlite3.connect(r"C:\Users\Mauro\OneDrive - АНО ВО Университет Иннополис\Документы\Innopolis\Empirical Methods\DataPrimaryProject\user1.sqlite")

In [3]:
signals_data = pd.read_sql_query("""SELECT * FROM Signals;""", con)
images_data = pd.read_sql_query("""SELECT * FROM Images;""", con)

In [4]:
filter_data = images_data.loc[(images_data["state"] == "Present") | (images_data["state"] == "Absent")]

filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])

sig_ti = signals_data.dateTime

<ipython-input-4-9e2eea3a92a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])


In [5]:
newfd = filter_data.filter(['id','dateTime','state'], axis=1)

newfd = newfd.reset_index(drop=True)

In [6]:
newfd

,id,dateTime,state
0,1,2021-03-30 13:13:55.899572,Absent
1,2,2021-03-30 13:13:57.749004,Present
2,4,2021-03-30 13:25:45.179473,Present
3,5,2021-03-30 13:25:47.124525,Absent
4,6,2021-03-30 13:25:47.511522,Present
...,...,...,...
1309,2514,2021-04-27 17:52:31.835608,Present
1310,2516,2021-04-27 17:54:50.824965,Present
1311,2518,2021-04-27 17:55:03.524461,Present
1312,2520,2021-04-27 17:55:46.828018,Present


In [7]:
pstates = list()
astates = list()

states = newfd.groupby([(newfd.state != newfd.state.shift()).cumsum()]) 

for i, g in states:
    if (len(g.dateTime.tolist()) > 1)  &  (g.state.all() == 'Present'):
        pstates.append((newfd.dateTime.iloc[g.index[0]],newfd.dateTime.iloc[g.index[0]+len(g)-1]))
    elif (g.state.all() == 'Absent') & (i > 1) & (i < len(states)-1):
        astates.append((newfd.dateTime.iloc[g.index[0]-1],newfd.dateTime.iloc[g.index[0]+len(g)]))

In [8]:
pre_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(pstates)):
    pre_intervals = list()
    for n in sig_ti:
        if (str(pstates[m][0]) <= n) &  (n <= str(pstates[m][1])):
           pre_intervals.append(n) 
    if len(pre_intervals) > 1:
       inter = pd.Series(pre_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    pre_sig.append(inter.iloc[-1] - inter.iloc[0])

In [9]:
abs_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(astates)):
    abs_intervals = list()
    for n in sig_ti:
        if (str(astates[m][0]) <= n) &  (n <= str(astates[m][1])):
           abs_intervals.append(n) 
    if len(abs_intervals) > 1:
       inter = pd.Series(abs_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    abs_sig.append(inter.iloc[-1] - inter.iloc[0])

In [11]:
pre_sig

[Timedelta('0 days 00:11:37.071785'),
 Timedelta('0 days 00:53:55.460654'),
 Timedelta('0 days 00:53:55.460654'),
 Timedelta('0 days 00:06:24.220788'),
 Timedelta('0 days 00:21:26.052278'),
 Timedelta('0 days 00:55:42.074310'),
 Timedelta('0 days 00:55:42.074310'),
 Timedelta('0 days 00:40:56.370002'),
 Timedelta('0 days 00:03:23.625458'),
 Timedelta('0 days 00:03:23.625458'),
 Timedelta('0 days 00:02:39.364621'),
 Timedelta('0 days 17:25:16.545503'),
 Timedelta('0 days 00:05:10.776724'),
 Timedelta('0 days 00:05:42.728046'),
 Timedelta('0 days 00:00:33.799997'),
 Timedelta('0 days 01:31:11.181822'),
 Timedelta('0 days 01:01:34.263441'),
 Timedelta('0 days 01:01:34.263441'),
 Timedelta('0 days 00:05:10.151673'),
 Timedelta('0 days 00:11:08.337134'),
 Timedelta('0 days 01:06:29.665499'),
 Timedelta('0 days 01:06:29.665499'),
 Timedelta('0 days 01:06:29.665499'),
 Timedelta('0 days 01:06:29.665499'),
 Timedelta('0 days 00:00:00.525863'),
 Timedelta('4 days 21:20:28.050726'),
 Timedelta('

In [12]:
abs_sig

[Timedelta('0 days 00:00:00.064959'),
 Timedelta('0 days 00:00:00.064959'),
 Timedelta('0 days 00:00:00.064959'),
 Timedelta('0 days 00:00:00.064959'),
 Timedelta('0 days 00:43:34.041046'),
 Timedelta('0 days 00:00:00.347200'),
 Timedelta('0 days 00:00:15.018512'),
 Timedelta('0 days 00:00:15.018512'),
 Timedelta('0 days 00:00:15.018512'),
 Timedelta('0 days 00:00:15.018512'),
 Timedelta('0 days 00:06:07.493201'),
 Timedelta('0 days 00:00:00.140006'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:28:23.693773'),
 Timedelta('0 days 00:10:49.979728'),
 Timedelta('0 days 00:10:49.979728'),
 Timedelta('0 days 00:04:58.204563'),
 Timedelta('0 days 00:04:58.204563'),
 Timedelta('0 days 00:06:06.967641'),
 Timedelta('0 days 00:04:21.695746'),
 Timedelta('

In [14]:
len(abs_sig)

138